# 도구 (Tools)

도구(Tool)는 유명한 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스임.

LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있고, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능함.

**LangChain 에 통합된 도구 리스트는 아래 링크에서 확인.**

- [LangChain 통합된 도구 리스트](https://python.langchain.com/docs/integrations/tools/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Agent-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
Agent-Tools


In [3]:
import warnings

warnings.filterwarnings("ignore")

## 빌트인 도구(built-in tools)

랭체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있음.

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있음.

관련 도구는 아래의 링크에서 참고

**참고**
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

### Python REPL 도구

이 도구는 Python 코드를 REPL(Read-Eval-Print Loop) 환경에서 실행하기 위한 클래스를 제공합니다
- PythonREPLTool

**설명**

- Python 셸 환경을 제공합니다.
- 유효한 Python 명령어를 입력으로 받아 실행합니다.
- 결과를 보려면 print(...) 함수를 사용해야 합니다.

**주요 특징**

- sanitize_input: 입력을 정제하는 옵션 (기본값: True)
- python_repl: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)

**사용 방법**

- PythonREPLTool 인스턴스 생성
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

**입력 정제**

- 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거합니다.

In [25]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

In [26]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

300



아래는 LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 반환하는 예제입니다.

**흐름 정리**
1. LLM 모델에게 특정 작업을 수행하는 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.


In [29]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = AzureChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [30]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 간단하게 작성하세요."))

CODE:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[11, 22, 30, 35, 39, 44]



### 검색 API 도구

Tavily 검색 API를 활용하여 검색 기능을 구현하는 도구입니다. 이 도구는 두 가지 주요 클래스를 제공합니다: `TavilySearchResults`와 `TavilyAnswer`.

**API 키 발급 주소**
- https://app.tavily.com/

발급한 API 키를 환경변수에 설정합니다.

`.env` 파일에 아래와 같이 설정합니다.

```
TAVILY_API_KEY=tvly-abcdefghijklmnopqrstuvwxyz
```

### TavilySearchResults

**설명**
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환합니다.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**
- `max_results` (int): 반환할 최대 검색 결과 수 (기본값: 5)
- `search_depth` (str): 검색 깊이 ("basic" 또는 "advanced")
- `include_domains` (List[str]): 검색 결과에 포함할 도메인 목록
- `exclude_domains` (List[str]): 검색 결과에서 제외할 도메인 목록
- `include_answer` (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- `include_raw_content` (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- `include_images` (bool): 쿼리 관련 이미지 목록 포함 여부

**반환 값**
- 검색 결과를 포함하는 JSON 형식의 문자열(url, content)

혹은 아래의 주석을 해제하고 발급받은 API 키를 입력합니다.

In [10]:
# import os

# os.environ["TAVILY_API_KEY"] = "TAVILY API 키 입력"

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)

In [33]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://wikidocs.net/book/14473',
  'content': '**랭체인(LangChain) 입문부터 응용까지** 이책은 거대언어모델(LLM)을 활용하여 애플리케이션과 파이프라인을 신속하게 구축할 때 주로 활용되는 랭체인(LangCha…'},
 {'url': 'https://wikidocs.net/book/2788',
  'content': '[DL 입문 🖋️] - 인공 신경망(Artificial Neural Network) - 유원준, March 30, 2024, 11:10 p.m. 오탈자 수정하였습니다. 15-03 허깅페이스 토크나이저(Huggingface Tokenizer) - 유원준, March 1, 2024, 5:12 p.m. https://wikidocs.net/99893 여기서는 멀쩡하니 여기서 보시면 될 것 같네요. 13-02 LSTM을 이용한 네이버 영화 리뷰 분류 - Aug. 24, 2024, 3:26 p.m. 이번에 사용할 데이터는 네이버 영화 리뷰 데이터입니다. 13-03 GRU를 이용한 IMDB 리뷰 분류하기 - Aug. 24, 2024, 3:22 p.m. 1D CNN을 이용하여 IMDB 영화 리뷰 데이터를 분류해보겠습니다. 20-05 BERT를 이용한 다중 레이블 분류 - 하나의 데이터에 다수의 레이블 - Aug. 24, 2024, 3:11 p.m. 한국어 BERT를 이용하여 다중 레이블 분류 문제를 풀어보겠습니다.'},
 {'url': 'https://wikidocs.net/book/4542',
  'content': '4, 2024, 10:10 p.m. set FLASK_APP=pybo 를 해줘도 아래 오류가 뜬다면 Error: Could not locate a Flask application. 3-13 마크다운 기능 적용하기 - 박응용, April 15, 2024, 3:12 p.m. 3-13 마크다운 기능 적용하기 - 송재영, April 15, 2024, 3 p.m. 플라스크 3.x 버전 

### Image 생성 도구 (DALL-E)

- `DallEAPIWrapper 클래스`: OpenAI의 DALL-E 이미지 생성기를 위한 래퍼(wrapper)입니다.

이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다. 다양한 설정 옵션을 통해 유연하고 강력한 이미지 생성 도구로 활용할 수 있습니다.

**주요 속성**

- `model`: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")

- `n`: 생성할 이미지 수 (기본값: 1)

- `size`: 생성할 이미지 크기
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style`: 생성될 이미지의 스타일 (기본값: "natural", "vivid")

- `quality`: 생성될 이미지의 품질 (기본값: "standard", "hd")

- `max_retries`: 생성 시 최대 재시도 횟수

**주요 기능**
- DALL-E API를 사용하여 텍스트 설명에 기반한 이미지 생성

**흐름 정리**

다음은 DALL-E Image Generator 를 사용하여 이미지를 생성하는 예제입니다.

이번에는 `DallEAPIWrapper` 를 사용하여 이미지를 생성해 보겠습니다.

이때 입력 프롬프트는 LLM 모델에게 이미지를 생성하는 프롬프트를 작성하도록 요청합니다.

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

# ChatOpenAI 모델 초기화
llm = AzureChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

# 이미지 프롬프트 출력
print(image_prompt)

Create a neo-classical painting that satirizes modern society's obsession with smartphones. The scene features a group of elegantly dressed individuals in a classical setting, reminiscent of ancient Rome or Greece. They are gathered around a large marble table, each one absorbed in their smartphones, completely ignoring their surroundings. The background features grand columns, lush greenery, and a serene landscape, contrasting with the modern devices in their hands. Include exaggerated facial expressions that convey distraction and bemusement. The color palette should be rich and earthy, with soft lighting that highlights the irony of their behavior. Add subtle details, like classical sculptures in the background, that emphasize the clash between tradition and modernity.


그럼, 이전에 생성한 이미지 프롬프트를 `DallEAPIWrapper` 에 입력하여 이미지를 생성해 보겠습니다.

**`DallEAPIWrapper` 에 대한 임시 버그 안내사항** (작성일: 2024-10-13)

- 현재 langchain 0.3.x 이상 버전에서 `DallEAPIWrapper` 에 대한 임시 버그가 있습니다. (`401 오류: invalid API key`)

따라서, 아래의 코드를 오류 없이 실행하기 위해서는 LangChain 버전을 0.2.16 으로 변경해야 합니다.

아래의 주석을 해제하고 실행하면 LangChain 버전을 0.2.16 으로 변경됩니다.

하지만, 이후 내용에서는 LangChain 버전을 0.3.x 이상으로 변경하여 사용하기 때문에

`poetry shell` 명령어를 통해 다시 최신 langchain 버전으로 변경해야 합니다.

이 과정이 번거로운 분들은 일단 `DallEAPIWrapper` 를 사용하지 않고 진행하셔도 무방합니다.

**업그레이드/다운그레이드** 후에는 반드시 상단 메뉴의 "Restart" 버튼을 클릭한 뒤 진행해야 합니다.

In [ ]:
# 임시 버전 다운그레이드 명령어 (실행 후 restart)
!pip install langchain==0.2.16 langchain-community==0.2.16 langchain-text-splitters==0.2.4 langchain-experimental==0.0.65 langchain-openai==0.1.20

In [ ]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image
import os

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

## 사용자 정의 도구(Custom Tool)

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다.

이를 위해서는 `langchain.tools` 모듈에서 제공하는 `tool` 데코레이터를 사용하여 함수를 도구로 변환합니다.

### @tool 데코레이터

이 데코레이터는 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.

**사용 방법**
1. 함수 위에 `@tool` 데코레이터 적용
2. 필요에 따라 데코레이터 매개변수 설정

이 데코레이터를 사용하면 일반 Python 함수를 강력한 도구로 쉽게 변환할 수 있으며, 자동화된 문서화와 유연한 인터페이스 생성이 가능합니다.

In [17]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [18]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [19]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

### 구글 뉴스기사 검색 도구

`langchain-teddynote` 패키지에서 제공하는 `GoogleNews` 도구를 사용하여 구글 뉴스기사를 검색하는 도구입니다.

**참고**
- API 키가 필요하지 않습니다. (RSS 피드를 사용하기 때문)

news.google.com 에서 제공하는 뉴스기사를 검색하는 도구입니다.

**설명**
- 구글 뉴스 검색 API를 사용하여 최신 뉴스를 검색합니다.
- 키워드를 기반으로 뉴스를 검색할 수 있습니다.
- 최신 뉴스를 검색할 수 있습니다.

**주요 매개변수**
- `k` (int): 반환할 최대 검색 결과 수 (기본값: 5)


사용하기 전 패키지를 업데이트 해주세요.

In [5]:
# !pip install -qU langchain-teddynote

In [34]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [35]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMikAFBVV95cUxNd2hPaGp0X2UwUl91TE1OUjhibDNDdUFCYWlCcUNxQkl4QnFDaFJaSldlN2ZoSjZTQXA5aHlQZDFsOGh2SUpBdVJ0UEVJMUtaVzhYU0daUXMzclkxYUE4Y3FoY2t1Q05fd0Q3TXpiSnRNZTA3dHphUU1LYjhPc3JNLVlrWk9pYUw2TGk4LWd2RXLSAaQBQVVfeXFMT3dwTmcweDJHRVY2ZDE4blFZSll5NWM1UEZ6VnBVU0kzRmVuRlJwM0hoTGdwdG00aUt6VkZ0QThMQzV5bk9VWEdEbVRHS0xhendxY2FtellsV3FWRXJYbGFFZXJPemlnTnViRFZ0ZHhMVGVlYkQ3Nm9DdXRibzhjVDY4UDlFMmhkWFpZdHl3bXFGRzZ2NmtlZGprSkxvQ3BqeDFzMTI?oc=5',
  'content': '[속보] 대통령 경호처 경찰 ‘서버 압수수색’ 거부 통보 - 조선일보'},
 {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE53Tkw0UEhmSzk3QnVyUVpCdUQ4dGpiOVFtQW41OFc3WnllT21zM1I5NE5IdTJZYTZaNGp1UXB4di1BaXhWaVJGdzAwZ3pPclRyU3lBc1V3?oc=5',
  'content': '‘어릴때 옆방쓰던 사이’…이재명, 권성동 만나 “누구 제거하려고 싸우면 본인도 불행” - 매일경제'},
 {'url': 'https://news.google.com/rss/articles/CBMid0FVX3lxTE1VRUc1akI0VzV1T2FxSTlaUHJ3TkNPd1ZrdTRRN29oTzBGN0k5ZEJEelI0cU13RG5ldHlEaUdTLW4tbGtkZk5ka2JZNm5mcFh2UzZaRkpvdTNQTGNXVEVQd2hNVFMyM0NnTW5EZzlKR1NJQXJaVjFv0gFmQVVf

In [36]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=3)

[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFBqYXhIWVZadlpNSjNsZlRXN0JLNU9tNWF6UGoycTVyMzRFbmRwVVV0VU8wOWpTbzd2NjNraXFPelhiZ09zVnFoWXQ5OVFFU2VaelZhUE1ZX3dpWTlpQXg2aGFBQldoNy1scGc?oc=5',
  'content': 'AI 투자 붐 타고 학술회의 \'뉴립스\'도 폭발적 성장..."핵심 논제는 추론" - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE1laTZOZGZzTmx1QWRrTUNHbXNQRHgtMG1jV0NUVC1ySFJueGRFaXZJa0NoQWI1dlJfSmxhd0pSMDU3UUpWeDRqOEk2aVdyckMwZUdIbXN30gFXQVVfeXFMTVZ4VERkaTJPaUhxaVVONms2XzNTT3lBOHZYNzM2ekgwUGFDajU5UTNELWVTV2tRZWJiT3VndTNtdDVvY3pSNW95emJlNmZPVHVoNVpfeVUw?oc=5',
  'content': "[투자의 창] 'AI 투자' 전성시대 - 서울경제신문"},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE4ybS1TVS1WQkFrZ3BUUzBKb1ZZcmlxTGdEQjhpbEJYa1NSeWFYTXp4djQzQlRFdk12eE9WQlEyaDI1ck1zNXpMZXhxeGxtMnF0ZnU0NG5kaUdMZ9IBVEFVX3lxTE1WaUdQbC1yYzUtTmNiY2k3NklUbVRkV05HMzdkaVVWTklvcHR0YTBEeko0MWFWa2ZUNnFiWEd5WVJqMzJKcWpocmlGTnpiblhOSnNLag?oc=5',
  'content': '"환율 오르는데 어디에 투자?"…신한은행, 최초 AI 상담 서비스 - 한국경제'}]

In [37]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [38]:
# 실행 결과
search_keyword.invoke({"query": "LangChain AI"})

LangChain AI


[{'url': 'https://news.google.com/rss/articles/CBMieEFVX3lxTE15VWVUMHAxYVdMcmpKWnhHLXVRZnh4LVB5cnd0Vldia2ZlRmNGMnByWVVlWXJMakhyZTJKVlJvMWRIRkRjVTFuUE52WU9oVnN2VkM1UTRnWThFLVRaaE04TDZKM0VzbmNJMkFLLWoyNzc0SktDVXJ4Rw?oc=5',
  'content': '가볍게 훑어보는 LangChain (랭체인) | 인사이트리포트 | 삼성SDS - Samsung SDS'},
 {'url': 'https://news.google.com/rss/articles/CBMizgJBVV95cUxQU1NSVHJ0eng3bzNVZWNibHBfV0txSkkxZDZvZXlsWXVhcHowWWlRVmwya1FnUm1Mc2FIWFFoSXdvdEY3WDVUbkprdnpDbFl2U1Fua3VjaEgtRHlZVnZlQjNIaTl6Um5NZ2dXenVQTjhMRUVDMzh2WUJjazloQUZmV1JfenBYT1EtWklqbkF2cEc2UFlwR1E0ek56Vjdqckc2Q0pESWd2NnlyYlQwcjBLNnpUem5CTzFaQUdWcTN4UUtROFJMV2ZvWXNrVm5hUDVLZ3cwTWZFT1lkZ2htWXl3LXZzQ25aeFI1SC02Ry1LQnU2aXZKTHEtNVZHazZnZHVBN1pSWEh6WFRoTGR2c19NOVBiXzFOUGNEQVRjSlRMRlBvY2lwaFlZcm1FOHNYZk1McVl5Unl3S1h0OHNadERXNVVuVU4yVVBsbV9HWG5n?oc=5',
  'content': '대규모 AI 배포: NVIDIA NIM과 LangChain이 AI 통합 및 성능을 혁신하는 방법 - Unite.AI'},
 {'url': 'https://news.google.com/rss/articles/CBMiiwFBVV95cUxNTF9wdzNISGRrdzdYbDFfeUhMVWFXNnlVQWg0aVAzMFBaOExPNDRZd